#  **Colab File for CNN training- *PyTorch***
Before you start please prepare the following:


1.   Uplaod the dataset as Zip file to GDrive
2.   Upload code files to GDrive



**Please MAKE sure that you are using GPU . Go to Runtime>> Change Runtime type  and select GPU**

#  **Connect to Google Drive**
 Allow Google Colab to connect to your GDrive

In [1]:

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%%capture
!pip install --upgrade gdown

In [ ]:
!nvidia-smi

Wed Dec 14 06:56:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 512.36       Driver Version: 512.36       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   41C    P8    11W /  N/A |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# **Prepare Dataset files and codes to run in Google Colab Instance**
1- Copy dataset file and codes to your Google Colab instance (Change file names if necessary ), dont have to copy the data files to the content here, as it will be copied in the next code

In [ ]:
%%capture
%cd /content/
!cp -av /content/gdrive/MyDrive/Bangla_sign_language/dataset/cnn_lstm/codes/* /content/

1- Copy the Data to the Virtual Machine

2- Unzip the dataset file in the instance,
   Please rename the downloaded file as Data.zip if it has a different name

In [ ]:
# Raw Dataset
%cd /content/
!gdown --id 1ZAgRgONITHweFELV8PAmV2kKdTeQPHdd
!jar -xvf Data.zip
!rm -rf Data.zip

# Install and Import Required Libraries (Conditionally Optional)
Use 'pip' to install libraries. Put '!' i.e. Exclamation Mark before each command, exclusive to Jupyter Notebooks. Some are already installed by Google Colab and others are required. If the IMPORT statements are present in the python code being run, no need to follow this step.

In [ ]:
!pip install torch==1.11.0 torchvision==0.12.0 torchaudio
!pip install torchvision
!pip install torchsummary
!pip install numpy
!pip install pandas
!pip install -U scikit-learn
!pip install scikit-image
!pip install ipython
!pip install Pillow
!pip install matplotlib
!pip install seaborn
!pip install gdown


In [2]:
# Printing out all outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
# PyTorch
import torch
from torchvision import transforms, models
from torch import optim, cuda, tensor
from torch.utils.data import DataLoader
import torch.nn as nn
# warnings
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
# Data science tools
import numpy as np
import os
from os import path
from importlib import import_module
# Visualizations
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 14
# customized functions
# import content
# from  content import utils
# from  utils import *

# GPU Checking (Optional)
Check the available GPU from the Google Server, GPU model and other related information.

In [3]:
import torch
print("Is CUDA enabled GPU Available?", torch.cuda.is_available())
print("Is GPU Initialized yet?", torch.cuda.is_initialized())
print("GPU Number:", torch.cuda.device_count())
print("Current GPU Index:", torch.cuda.current_device())
print("GPU Type:", torch.cuda.get_device_name(device=None))
print("GPU Capability:", torch.cuda.get_device_capability(device=None))

# Mean Std

**Raw Data**\
Train\
mean= [0.6125, 0.5889, 0.5719], std = [0.2114, 0.2359, 0.2421]\
Test\
mean= [0.5921, 0.5725, 0.5579], std = [0.1909, 0.2126, 0.2169]\
Val\
mean= [0.653, 0.6489, 0.6347], std = [0.2044, 0.2064, 0.2096]\
mean:[0.6192     0.60343333 0.58816667]
std:[0.20223333 0.2183     0.22286667]

**Segmented Data**\
Train\
mean= [0.0624, 0.0547, 0.052], std = [0.1926, 0.1781, 0.173]\
Test\
mean= [0.0668, 0.0579, 0.0552], std = [0.1967, 0.1806, 0.1756]\
Val\
mean= [0.0673, 0.06, 0.0572], std = [0.1963, 0.1856, 0.181]\
mean:[0.0655     0.05753333 0.0548    ]
std:[0.1952     0.18143333 0.17653333]

**Pose Data**\
Train\
mean= [0.0089, 0.0064, 0.0066], std = [0.078, 0.0632, 0.0636]\
Test\
mean= [0.0092, 0.0066, 0.0069], std = [0.0789, 0.0645, 0.0648]\
Val\
mean= [0.0095, 0.0069, 0.0072], std = [0.0803, 0.0661, 0.0664]\
mean:[0.0092     0.00663333 0.0069    ]
std:[0.07906667 0.0646     0.06493333]


# **CNN Training**
1- CNN configuration (Model, epochs, etc..)

**Current Supported Netwroks**
check available_models.txt for the full list
---


model -......................input size

                      
*  'squeezenet1_0'--------224
*  'vgg16_bn'-----------------224    
*   'vgg19_bn'-----------------224
*   'resnet18'------------------224
*   'resnet50'------------------224
*   'resnet101'----------------224
*   'resnet151'----------------224
*  'inception_v3' -----------299
*   'inceptionresnetv2'----299
*   'xception'-------------------299
*   'chexnet'--------------------224
*   'densenet121'------------224
*   'densenet161'------------224
*   'densenet201'------------224



In [ ]:
%cd /content/

/content


In [ ]:
#rename the folder,change mean and std dev
%%writefile config.py
# CNN configuration file

##### DO NOT EDIT THESE LINES #####
config = {}
###################################


#### START EDITING FROM HERE ######
config['parentdir'] = '/content/'                                                   # main directory
config['input_type'] = 'video'                         # 'NIFTI' or 'video'
config['ImageNet'] = True                              # set to 'True' to use ImageNet weights or set to 'False' to train from scratch
config['q_order'] = 1                                 # qth order Maclaurin approximation, common values: {1,3,5,7,9}. q=1 is equivalent to conventional CNN
config['ONN'] = False                                 # set to 'True' if you are using ONN
config['input_ch'] = 3                                 # 1 for gray scale x-rays, and 3 for RGB (3channel) x-rays
config['frames'] = 60                                  #
config['freeze_CNN'] = False                          # set to true to freeze CNN layers
config['LSTM_hidden_size'] = 256                       # number of hidden size of LSTM layers 128, 256, 512, 1024
config['LSTM_num_layers'] = 3                          # number of LSTM layers
config['LSTM_drop_rate'] = 0.3                         # LSTM drop out rate of LSTM layers except last layer
config['bidirectional']=False
config['batch_size']  = 3                              # batch size, Change to fit hardware
config['input_mean'] =[0.46096033 ,0.52650049 ,0.53160196]# Dataset mean per channel, b/w [0,1]
config['input_std'] =[0.24271709, 0.21365579, 0.21639321]                              # Dataset std per channel,  b/w [0,1]
config['optim_fc'] = 'Adam'                            # 'Adam' or 'SGD'
config['lr'] = 0.0001                                    # learning rate
config['stop_criteria'] = 'accuracy'                       # Stopping criteria: 'loss' or 'accuracy'
config['n_epochs']  = 60                               # number of training epochs
config['epochs_patience'] = 60                          # if val loss did not decrease for a number of epochs then decrease learning rate by a factor of lr_factor
config['lr_factor'] = 0.2
config['max_epochs_stop'] = 60                        # maximum number of epochs with no improvement in validation loss for early stopping
config['num_folds']  = 1                               # number of cross validation folds
config['Resize_h'] = 120                               # network input size
config['Resize_w'] = config['Resize_h']

#config['load_model'] = '/content/checkpoint_fold_1.pt'
#config['load_model'] = '/content/gdrive/MyDrive/Bangla_sign_language/dataset/cnn_lstm/Results/mobilenet_v2_q1_fixF/checkpoint_fold_1.pt' # specify path of pretrained model wieghts or set to False to train from scratch
config['load_model'] = False                                                       # specify path of pretrained model wieghts or set to False to train from scratch
config['model_name'] = 'mobilenet_v2_q1_fixFF'
                                             # choose a unique name for result folder
config['model_to_load'] = 'mobilenet_v2'                                               # chosse one of the following models:
#  'squeezenet1_0', 'vgg16', 'vgg16_bn', 'vgg19', 'vgg19_bn', 'resnet18', 'resnet50', 'resnet101', 'resnet152', 'inception_v3', 'inceptionresnetv2'
#  'xception', 'chexnet', 'nasnetalarge', 'pnasnet5large', 'densenet121', 'densenet161', 'densenet201', 'shufflenet' , 'googlenet', 'mobilenet_v2', 'nasnetamobile', 'alexnet'
config['encoder'] = False  # set to 'True' if you retrain Seg. model encoder as a classifer
config['fold_to_run'] = [1,1] # define as [] to loop through all folds, or specify start and end folds i.e. [3, 5] or [5, 5]
##################

##################
# config['Results_path'] = '/content/gdrive/MyDrive/KArSL/Fixed/Segmented/Results'                 # main results file
config['Results_path'] =  '/content/gdrive/MyDrive/My_slient_speech/dataset/cnn_lstm/Results'
config['save_path'] = config['Results_path'] +'/'+ config['model_name']              # save path
##################



Writing config.py


2- Start training

In [4]:
#rename data folder
%cd /content/

!python TrainCNN.py

In [ ]:
%cd /content/

/content


In [ ]:
%%writefile config_inference.py
# CNN test configuration file

##### DO NOT EDIT THESE LINES #####
config = {}
###################################

#### START EDITING FROM HERE ######
config['parentdir'] =  '/content/'                                           # main directory
config['input_type'] = 'video'                         # 'NIFTI' or 'video'
config['ONN'] = False                                   # set to 'True' if you are using ONN
config['input_ch'] = 3                                  # 1 for gray scale x-rays, and 3 for RGB (3channel) x-rays
config['batch_size']  = 3                               # batch size, Change to fit hardware
config['num_folds']  = 5                                # number of cross validation folds
config['CI']  = 0.9                                     # Confidence interval (missied cases with probability>=CI will be reported in excel file)
config['input_mean'] = [0.28443955, 0.23666358, 0.21275447]                              # Dataset mean per channel, b/w [0,1]
config['input_std'] = [0.26398318, 0.23218298, 0.23881866]                             # Dataset std per channel,  b/w [0,1]
config['Resize_h'] = 224                                # network input size
config['Resize_w'] = config['Resize_h']
# config['load_model'] = config['parentdir'] + 'load_model/K_Unet_CXR.pt'    # specify full path of pretrained model pt file
config['load_model'] = False                                                 # or set to False to load trained model by train code
config['labeled_Data'] = True                                                # set to true if you have the labeled test set
config['model_name'] = 'mobilenet_v2_q1'                                    # name of trained model .pt file, same name used in train code
config['new_name'] = 'mobilenet_v2_q1_test'                                      # specify a new folder name to save test results,
config['fold_to_run'] = [1,1] # define as [] to loop through all folds, or specify start and end folds i.e. [3, 5] or [5, 5]
##################
##################

##################
config['Results_path'] = '/content/gdrive/MyDrive/Sumon_Storage/SUbjper5cls/Results'         # main results file
if config['new_name']:
    config['save_path'] = config['Results_path'] +'/'+ config['new_name']    # new save path
else:
    config['save_path'] = config['Results_path'] +'/'+ config['model_name']  # same save path used for training
##################

Overwriting config_inference.py


In [ ]:
%cd /content/
!python Inference.py

/content
Train on gpu: True
1 gpus detected.
#############################################################
started fold 1
model device: cuda
Test Loss: 0.0398,  Test Accuracy: 98.52%
completed fold 1
#############################################################


        kharap  ...  maa
kharap      30  ...    0
baba         0  ...    0
onek         0  ...    0
gorom        0  ...    0
tumi         0  ...    0
kom          0  ...    1
khabo        1  ...    0
bhalo        0  ...    0
maa          0  ...   30

[9 rows x 9 columns]


                  Accuracy  ...  Specificity
kharap               99.63  ...        99.58
baba                100.00  ...       100.00
onek                100.00  ...       100.00
gorom                99.63  ...        99.58
tumi                 99.63  ...       100.00
kom                  99.26  ...        99.58
khabo                99.26  ...       100.00
bhalo               100.00  ...       100.00
maa                  99.63  ...        99.58
Weighted Ave

In [ ]:
!cp -av /content/gdrive/MyDrive/KArSL/Fixed/Segmented/Results/resnet18_self_20_v2/AccuracyPerEpoch_fold_1.png /content/gdrive/MyDrive/KArSL/Fixed/Segmented/Results_Best/resnet18_self_10
!cp -av /content/gdrive/MyDrive/KArSL/Fixed/Segmented/Results/resnet18_self_20_v2/LossPerEpoch_fold_1.png /content/gdrive/MyDrive/KArSL/Fixed/Segmented/Results_Best/resnet18_self_10
!cp -av /content/gdrive/MyDrive/KArSL/Fixed/Segmented/Results/resnet18_self_20_v2/resnet18_self_20_v2_fold_1.pt /content/gdrive/MyDrive/KArSL/Fixed/Segmented/Results_Best/resnet18_self_10
!cp -av /content/gdrive/MyDrive/KArSL/Fixed/Segmented/Results/resnet18_self_20_v2/resnet18_self_20_v2_test_fold_1.pt /content/gdrive/MyDrive/KArSL/Fixed/Segmented/Results_Best/resnet18_self_10

In [ ]:
%cd /content/
!python InferenceTime.py

/content
0.12451755074600578


In [ ]:
%cd /content/
!python InferenceTime.py

/content
0.25646495399898595
